I divide the seq into 7 networks

In [13]:
import sys
sys.path.append("../mypkg")

In [14]:
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT
from utils.misc import load_pkl, save_pkl, get_ball_cor
from utils.colors import qual_cmap

In [15]:
%load_ext autoreload
%autoreload 2
# 0,1, 2, 3, be careful about the space

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
import scipy
from scipy.io import loadmat
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import bct # the pkg to get graph features
from joblib import Parallel, delayed
from easydict import EasyDict as edict
import pandas as pd

plt.style.use(FIG_ROOT/'base.mplstyle')

# Fns and params

## Some fns

In [17]:
def _preprocess_ts(ts):
    """preprocessing, 
        1. detrend
        2. resample (len of seq has 235 or 555, I make it consistent to 235)
    """
    ts = signal.detrend(ts);
    if ts.shape[-1] > 235:
        ts = signal.resample(ts, num=235, axis=-1)
    return ts
    

    
def _load_data(n):
    """Sub idx is from 1, n: the sub idx
    """
    return mat_data['ts_321_273'][np.where(mat_data['subj_321_ts'][:, 0] == n)[0], :].transpose()

## Load data and params

In [18]:
# load data
mat_data = loadmat(DATA_ROOT/"ad_ftd_hc_fmri_data.mat");

# some parameters

num_rois = 246 # the BNA has 246 regions
num_sps = 321

# get the 7 nets index
import pandas as pd
labs = pd.read_csv(DATA_ROOT/"BN246_RegionList.csv");
yeo_7nets_lab = np.array(labs["Yeo_7network"][:246]);

yeo_7nets_dict = pd.read_csv(DATA_ROOT/"yeo_7nets_dict.txt");
yeo_7nets_dict = {yeo_7nets_dict["lab"][ix]:"_".join(yeo_7nets_dict["name"][ix].strip().split(" "))
                  for ix in range(7)}

{1: 'Visual',
 2: 'Somatomotor',
 3: 'Dorsal_Attention',
 4: 'Ventral_Attention',
 5: 'Limbic',
 6: 'Frontoparietal',
 7: 'Default'}

In [44]:
jesse_org_order = np.loadtxt(DATA_ROOT/"jesse_org_order.txt", dtype=str, delimiter=",")[:246];
my_file_order = np.array(labs["Unnamed: 0"][:246]);
df = pd.DataFrame([jesse_org_order, my_file_order]).T;
df.columns = ["Jesse's", "BN official web"];

# Get the data

divide the ts into 7 nets (with detrend and resample)

In [7]:
from collections import defaultdict as ddict
sub_ix = 1
yeo_7nets_ts = ddict(list)
for sub_ix in range(1, num_sps+1):
    ts = _load_data(sub_ix)[:num_rois]
    ts = _preprocess_ts(ts)
    for net_ix in range(1, 8):
        yeo_7nets_ts[yeo_7nets_dict[net_ix]].append(ts[yeo_7nets_lab==net_ix])

In [8]:
save_pkl(DATA_ROOT/"ad_ftd_hc_fmri_data_7nets.pkl", yeo_7nets_ts)

Save to /data/rajlab1/user_data/jin/MyResearch/jesse_data/notebooks/../mypkg/../data/ad_ftd_hc_fmri_data_7nets.pkl
